# General DA fittings and Fokker-Planck comparisons


## Setup scripts if we are under SWAN

In [ ]:
# Working in the right path
%cd /eos/project/d/da-and-diffusion-studies/DA_Studies/Simulations/Models/da_sixtrack

In [ ]:
# Install the libraries
import sys
!{sys.executable} -m pip install --user tqdm pynverse sixtrackwrap crank_nicolson_numba
!export PYTHONPATH=$CERNBOX_HOME/.local/lib/python3.7/site-packages:$PYTHONPATH

In [ ]:
# For this "presentation" only!
import warnings
warnings.filterwarnings('ignore')

## Imports

In [1]:
%matplotlib widget

In [2]:
# Base libraries
import math
import numpy as np
import scipy.integrate as integrate
from scipy.special import erf
import pickle
import itertools
from scipy.optimize import curve_fit

from numba import njit, prange

# Personal libraries
#import sixtrackwrap_light as sx
import sixtrackwrap as sx

from tqdm.notebook import tqdm
import time
import matplotlib.pyplot as plt
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.ticker as ticker
from math import gcd

import pandas as pd

from scipy.special import lambertw
from scipy.interpolate import interp1d

import warnings

## Load data and setup original DA

In [3]:
savepath = "data/"
raw_engine = sx.radial_scanner.load_values(savepath + "big_scan.pkl")
engine = sx.uniform_radial_scanner.load_values(savepath + "big_scan.pkl")

min_turns = engine.min_time
max_turns = engine.max_time
n_turn_samples = 500

turn_sampling = np.linspace(min_turns, max_turns, n_turn_samples, dtype=np.int_)[::-1]

d_r = engine.dr
starting_step = engine.starting_step

# BASELINE COMPUTING
baseline_samples = 33
baseline_total_samples = baseline_samples ** 3

In [4]:
alpha_preliminary_values = np.linspace(-1.0, 1.0, baseline_samples)
alpha_values = np.arccos(alpha_preliminary_values) / 2
theta1_values = np.linspace(0.0, np.pi * 2.0, baseline_samples, endpoint=False)
theta2_values = np.linspace(0.0, np.pi * 2.0, baseline_samples, endpoint=False)

d_preliminar_alpha = alpha_preliminary_values[1] - alpha_preliminary_values[0]
d_theta1 = theta1_values[1] - theta1_values[0]
d_theta2 = theta2_values[1] - theta2_values[0]

alpha_mesh, theta1_mesh, theta2_mesh = np.meshgrid(alpha_values, theta1_values, theta2_values, indexing='ij')

alpha_flat = alpha_mesh.flatten()
theta1_flat = theta1_mesh.flatten()
theta2_flat = theta2_mesh.flatten()

In [5]:
radiuses = raw_engine.extract_DA(turn_sampling)
radiuses = radiuses.reshape((baseline_samples, baseline_samples, baseline_samples, len(turn_sampling)))

In [6]:
DA = []

mod_radiuses = radiuses.copy()
mod_radiuses = np.power(radiuses, 4)
mod_radiuses1 = integrate.simps(mod_radiuses, x=theta1_values, axis=1)
mod_radiuses2 = integrate.simps(mod_radiuses1, x=theta2_values, axis=1)
mod_radiuses3 = integrate.simps(mod_radiuses2, x=alpha_preliminary_values, axis=0)

for i in range(len(turn_sampling)):
    DA.append(
        np.power(
            mod_radiuses3[i] / (2 * theta1_values[-1] * theta2_values[-1]),
            1/4
        )
    )

DA = np.asarray(DA)

In [7]:
axis_sampling = np.concatenate((turn_sampling,[0.0]))

## Prepare inverse functions for obtaining DA from loss values

In [8]:
from pynverse import inversefunc

# Uniform 4D distribution

@njit
def uniform_loss(DA, DA0):
    return (DA ** 4 / DA0 ** 4)

@njit
def DA_from_unifom_loss(loss, DA0):
    return np.power((loss * DA0 ** 4), 1/4)

# Symmetric 4D gaussian

@njit
def symmetric_gaussian_loss(DA, sigma, DA0):
    baseline = - np.exp(- 0.5 * (DA0 / sigma) ** 2) * (DA0 ** 2 + 2 * sigma ** 2) + 2 * sigma ** 2
    return (- np.exp(- 0.5 * (DA / sigma) ** 2) * (DA ** 2 + 2 * sigma ** 2) + 2 * sigma ** 2) / baseline


def DA_from_symmetric_gaussian_loss(loss, sigma, DA0):
    func = inversefunc(
        lambda x: symmetric_gaussian_loss(x, sigma, DA0),
        domain=[0.0, DA0]
    )
    return func(loss)
    

## Define Fitting Models!

### $\chi^2$ function

In [9]:
def chi_2(original, estimate):
    return np.sum(np.power(estimate - original, 2) / original)

### Model 2

$$D(N) = \rho_\ast \left(\frac{\kappa}{2e}\right)^\kappa \frac{1}{\ln^\kappa\frac{N}{N_0}}$$


In [10]:
@njit
def model_2(x, rho, n0, k):
    return rho * np.power(k / (2 * np.exp(1)), k) / (np.power(np.log(x / n0), k))


def explore_k_model_2(turns, da, k_min, k_max, n_samples):
    ks = np.linspace(k_min, k_max, n_samples)
    pars = []
    errs = []
    co_pars = []
    for k in tqdm(ks):
        par, co_par = curve_fit(
            lambda x, a, b : model_2(x, a, b, k),
            turns,
            da,
            bounds=([0, 0.00001],[np.inf, turns[-1]-0.0001])
        )
        pars.append(par)
        co_pars.append(co_par)
        errs.append(chi_2(DA, model_2(turn_sampling, par[0], par[1], k)))
    return np.asarray(pars), np.asarray(errs), np.asarray(co_pars)

### Model 4

$$D(N)=\rho_{*} \times \frac{1}{\left[-2 \mathrm{e} \lambda \mathcal{W}_{-1}\left(-\frac{1}{2 \mathrm{e} \lambda}\left(\frac{\rho_{\ast}}{6}\right)^{1 / \kappa}\left(\frac{8}{7} N\right)^{-1 /(\lambda \kappa)}\right)\right]^{\kappa}}
$$

(with $\lambda=0.5$)

In [11]:
def model_4(x, rho, k):
    lamb = 0.5
    return np.real(
        rho
        / np.power(-2 * np.exp(1) * lamb * lambertw(- (1 / (2 * np.exp(1) * lamb)) * np.power(rho / 6, 1 / k) * np.power((8/7) * x, -1 / (lamb * k)), -1), k)
    )

def explore_k_model_4(turns, da, k_min, k_max, n_samples):
    ks = np.linspace(k_min, k_max, n_samples)
    pars = []
    errs = []
    co_pars = []
    for k in tqdm(ks):
        par, co_par = curve_fit(
            lambda x, a : model_4(x, a, k),
            turns,
            da,
            bounds=([0.1],[np.inf])
        )
        pars.append(par)
        co_pars.append(co_par)
        errs.append(chi_2(DA, model_4(turn_sampling, par[0], k)))
    return np.asarray(pars), np.asarray(errs), np.asarray(co_pars)

### A more free Model 4

$$D(N)=\rho_{*} \times \frac{1}{\left[-2 \mathrm{e} \lambda \mathcal{W}_{-1}\left(-\frac{1}{\lambda\kappa}\left(\frac{N}{N_0}\right)^{-\frac{1}{\lambda\kappa}}\right)\right]^{\kappa}}
$$

(with $\lambda=0.5$)

In [12]:
def model_4_free(x, rho, n0, k):
    lamb = 0.5
    temp = (
        rho / (np.power(- 2 * lamb * np.exp(1) * lambertw(- (1 / (lamb * k)) * np.power(x / n0, - 1 / (lamb * k)), -1), k))
    )
    
    return np.real(temp) # TODO: FIX IT WITH PROPER MATH!!!


def explore_model_4_free(turns, da, k_min, k_max, k_samples):
    ks = np.linspace(k_min, k_max, k_samples)
    pars = []
    errs = []
    co_pars = []
    for k in tqdm(ks):
        par, co_par = curve_fit(
            lambda x, a, b : model_4_free(x, a, b, k),
            turns,
            da,
            bounds=([0.01, 0.01], [np.inf, turns[-1] - 0.1])
        )
        pars.append(par)
        co_pars.append(co_par)
        errs.append(chi_2(DA, model_4_free(turn_sampling, par[0], par[1], k)))
    return np.asarray(pars), np.asarray(errs), np.asarray(co_pars)

## Visualize original DA value from data

In [13]:
fig1, ax1 = plt.subplots()
ax1.plot(turn_sampling, DA)
ax1.set_title("Original $DA$ values (LHC on SixTrack)")
ax1.set_xlabel("$N$ turns")
ax1.set_ylabel("$DA(N)$")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$DA(N)$')

## Setup Dataframe for storing the fitting data properly!

In [14]:
labels = (
    ("type", ""),
    ("sigma", ""),
    ("Model 2", "k"),
    ("Model 2", "k err"),
    ("Model 2", "rho"),
    ("Model 2", "rho err"),
    ("Model 2", "N0"),
    ("Model 2", "N0 err"),
    ("Model 2", "Chi2"),
    ("Model 4 (2 free pars)", "k"),
    ("Model 4 (2 free pars)", "k err"),
    ("Model 4 (2 free pars)", "rho"),
    ("Model 4 (2 free pars)", "rho err"),
    ("Model 4 (2 free pars)", "lambda"),
    ("Model 4 (2 free pars)", "Chi2"),
    ("Model 4 (3 free pars)", "k"),
    ("Model 4 (3 free pars)", "k err"),
    ("Model 4 (3 free pars)", "rho"),
    ("Model 4 (3 free pars)", "rho err"),
    ("Model 4 (3 free pars)", "N0"),
    ("Model 4 (3 free pars)", "N0 err"),
    ("Model 4 (3 free pars)", "lambda"),
    ("Model 4 (3 free pars)", "Chi2"),
)

fitting_data = pd.DataFrame(columns=pd.MultiIndex.from_tuples(labels))

## How is the error on the DA loss computed right now?

1. Consider all the radiuses sampled.
2. Compute the DA value.
3. For every radius sampled, compute the difference from the DA value.
4. The absolute value of the average of all these differences is considered as error.

(I tried using the Standard Deviation of the radiuses distribution, but it ended up being 10% of the DA itself, so we "need" somehow a smaller error estimation)
## Loss and Fits -- Uniform Beam Distribution
### Loss comparison

In [15]:
# Set DA0 cutting point
DA0 = 32.0

# Assign uniform weights to engine and compute loss
engine.assign_weights(
    sx.assign_uniform_distribution()
)
real_values = engine.compute_loss(turn_sampling, DA0)
real_values = np.concatenate((real_values, [1.0]))

# Compute DA-based loss
values = uniform_loss(DA, DA0)
values = np.concatenate((values, [1.0]))

# Error computing
values1 = uniform_loss(DA - np.absolute(np.mean(radiuses - DA, axis=(0,1,2))), DA0)
values1 = np.concatenate((values1, [1.0]))

values2 = uniform_loss(DA + np.absolute(np.mean(radiuses - DA, axis=(0,1,2))), DA0)
values2 = np.concatenate((values2, [1.0]))

#### Visualize Loss comparison

In [16]:
fig2, ax2 = plt.subplots()
ax2.plot(axis_sampling, values, label="Values from DA")
ax2.fill_between(axis_sampling, values1, values2, label="Values from DA - error", color="C0", alpha=0.4)
ax2.plot(axis_sampling, real_values, label="Values from weights")
ax2.legend()
ax2.set_xlabel("$N$ turns")
ax2.set_ylabel("Active beam")
ax2.set_title("Uniform beam (Cutting Point at $DA=26.0$)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Uniform beam (Cutting Point at $DA=26.0$)')

### Get DA from loss

In [17]:
DA_from_loss = DA_from_unifom_loss(real_values, DA0)

#### Visualize difference

In [18]:
fig3, ax3 = plt.subplots()
ax3.plot(turn_sampling, DA, label="Original DA")
ax3.plot(axis_sampling[:-1], DA_from_loss[:-1], label="DA extracted from real loss values")
ax3.legend()
ax3.set_xlabel("$N$ turns")
ax3.set_ylabel("$DA(N)$")
ax3.set_title("Comparison between original DA and DA obtained from real Loss")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Comparison between original DA and DA obtained from real Loss')

### Compare fitting values with Model 2

In [19]:
k_min = 0.05
k_max = 0.20
samples = 400

ks = np.linspace(k_min, k_max, samples)

real_pars, real_errs, real_co_pars = explore_k_model_2(turn_sampling, DA, k_min, k_max, samples)
loss_pars, loss_errs, loss_co_pars = explore_k_model_2(axis_sampling[:-1], DA_from_loss[:-1], k_min, k_max, samples)

real_selected_err_2 = np.min(real_errs)
real_selected_k_2 = ks[np.argmin(real_errs)]
real_selected_pars_2 = real_pars[np.argmin(real_errs)]
real_selected_co_pars_2 = real_co_pars[np.argmin(real_errs)]

loss_selected_err_2 = np.min(loss_errs)
loss_selected_k_2 = ks[np.argmin(loss_errs)]
loss_selected_pars_2 = loss_pars[np.argmin(loss_errs)]
loss_selected_co_pars_2 = loss_co_pars[np.argmin(loss_errs)]

In [20]:
print("REAL:", "$\\kappa:$", real_selected_k_2, ", $\\rho_\\ast$ and $N_0$", real_selected_pars_2)
print("FROM LOSS:", "$\\kappa:$", loss_selected_k_2, ", $\\rho_\\ast$ and $N_0$", loss_selected_pars_2)

REAL: $\kappa:$ 0.11992481203007521 , $\rho_\ast$ and $N_0$ [36.15442002 46.76235005]
FROM LOSS: $\kappa:$ 0.12142857142857144 , $\rho_\ast$ and $N_0$ [37.48209719 57.0904288 ]


In [21]:
fig3, ax3 = plt.subplots()

ax3.plot(turn_sampling, DA, label="Original DA")
ax3.plot(turn_sampling, model_2(turn_sampling, real_selected_pars_2[0], real_selected_pars_2[1], real_selected_k_2), label="Fitting on original DA\n$\\kappa = {:.4f},\\rho_\\ast = {:.4f}, N_0={:.4f}$".format(real_selected_k_2, real_selected_pars_2[0], real_selected_pars_2[1] ))

ax3.plot(axis_sampling[:-1], DA_from_loss[:-1], label="DA extracted from real loss values")
ax3.plot(turn_sampling, model_2(turn_sampling, loss_selected_pars_2[0], loss_selected_pars_2[1], loss_selected_k_2), label="Fitting on original DA\n$\\kappa = {:.4f},\\rho_\\ast = {:.4f}, N_0={:.4f}$".format(loss_selected_k_2, loss_selected_pars_2[0], loss_selected_pars_2[1] ))

ax3.legend()
ax3.set_xlabel("$N$ turns")
ax3.set_ylabel("$DA(N)$")
ax3.set_title("Comparison between original DA and DA obtained from real Loss, with Model 2 Fits")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Comparison between original DA and DA obtained from real Loss, with Model 2 Fits')

### Compare fitting values with Model 4

In [22]:
k_min = 0.15
k_max = 0.30
samples = 400

ks = np.linspace(k_min, k_max, samples)

real_pars, real_errs, real_co_pars = explore_k_model_4(turn_sampling, DA, k_min, k_max, samples)
loss_pars, loss_errs, loss_co_pars = explore_k_model_4(axis_sampling[:-1], DA_from_loss[:-1], k_min, k_max, samples)

real_selected_err_4 = np.min(real_errs)
real_selected_k_4 = ks[np.argmin(real_errs)]
real_selected_pars_4 = real_pars[np.argmin(real_errs)]
real_selected_co_pars_4 = real_co_pars[np.argmin(real_errs)]

loss_selected_err_4 = np.min(loss_errs)
loss_selected_k_4 = ks[np.argmin(loss_errs)]
loss_selected_pars_4 = loss_pars[np.argmin(loss_errs)]
loss_selected_co_pars_4 = loss_co_pars[np.argmin(loss_errs)]

In [23]:
print("REAL:", "$\\kappa:$", real_selected_k_4, ", $\\rho_\\ast$ and $N_0$", real_selected_pars_4)
print("FROM LOSS:", "$\\kappa:$", loss_selected_k_4, ", $\\rho_\\ast$ and $N_0$", loss_selected_pars_4)

REAL: $\kappa:$ 0.21616541353383456 , $\rho_\ast$ and $N_0$ [60.22905065]
FROM LOSS: $\kappa:$ 0.22368421052631576 , $\rho_\ast$ and $N_0$ [64.37611132]


In [24]:
fig3, ax3 = plt.subplots()

ax3.plot(turn_sampling, DA, label="Original DA")
ax3.plot(turn_sampling, model_4(turn_sampling, real_selected_pars_4[0], real_selected_k_4), label="Fitting on original DA\n$\\kappa = {:.4f},\\rho_\\ast = {:.4f}$".format(real_selected_k_4, real_selected_pars_4[0]))

ax3.plot(axis_sampling[:-1], DA_from_loss[:-1], label="DA extracted from real loss values")
ax3.plot(turn_sampling, model_4(turn_sampling, loss_selected_pars_4[0], loss_selected_k_4), label="Fitting on original DA\n$\\kappa = {:.4f},\\rho_\\ast = {:.4f}$".format(loss_selected_k_4, loss_selected_pars_4[0]))

ax3.legend()
ax3.set_xlabel("$N$ turns")
ax3.set_ylabel("$DA(N)$")
ax3.set_title("Comparison between original DA and DA obtained from real Loss,\n with Model 4 Fits $(\\lambda=0.5)$, 2 free parameters")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Comparison between original DA and DA obtained from real Loss,\n with Model 4 Fits $(\\lambda=0.5)$, 2 free parameters')

### Compare fitting values with Model 4 (But more free)

In [25]:
k_min = 0.05
k_max = 0.20
samples = 400

ks = np.linspace(k_min, k_max, samples)

real_pars, real_errs, real_co_pars = explore_model_4_free(turn_sampling[:], DA[:], k_min, k_max, samples)
loss_pars, loss_errs, loss_co_pars = explore_model_4_free(axis_sampling[:-1], DA_from_loss[:-1], k_min, k_max, samples)

real_selected_err_4_free = np.min(real_errs)
real_selected_k_4_free = ks[np.argmin(real_errs)]
real_selected_pars_4_free = real_pars[np.argmin(real_errs)]
real_selected_co_pars_4_free = real_co_pars[np.argmin(real_errs)]

loss_selected_err_4_free = np.min(loss_errs)
loss_selected_k_4_free = ks[np.argmin(loss_errs)]
loss_selected_pars_4_free = loss_pars[np.argmin(loss_errs)]
loss_selected_co_pars_4_free = loss_co_pars[np.argmin(loss_errs)]

In [26]:
print("REAL:", "$\\kappa:$", real_selected_k_4_free, ", $\\rho_\\ast$ and $N_0$", real_selected_pars_4_free)
print("FROM LOSS:", "$\\kappa:$", loss_selected_k_4_free, ", $\\rho_\\ast$ and $N_0$", loss_selected_pars_4_free)

REAL: $\kappa:$ 0.12293233082706768 , $\rho_\ast$ and $N_0$ [36.79845342 43.27224119]
FROM LOSS: $\kappa:$ 0.12406015037593987 , $\rho_\ast$ and $N_0$ [38.08575855 52.81558307]


In [27]:
fig3, ax3 = plt.subplots()

ax3.plot(turn_sampling, DA, label="Original DA")
ax3.plot(turn_sampling, model_4_free(turn_sampling, real_selected_pars_4_free[0], real_selected_pars_4_free[1], real_selected_k_4_free), label="Fitting on original DA\n$\\kappa = {:.4f},\\rho_\\ast = {:.4f}, N_0 = {:.4f}$".format(real_selected_k_4_free, real_selected_pars_4_free[0], real_selected_pars_4_free[1]))

ax3.plot(axis_sampling[:-1], DA_from_loss[:-1], label="DA extracted from real loss values")
ax3.plot(turn_sampling, model_4_free(turn_sampling, loss_selected_pars_4_free[0], loss_selected_pars_4_free[1], loss_selected_k_4_free), label="Fitting on original DA\n$\\kappa = {:.4f},\\rho_\\ast = {:.4f}, N_0 = {:.4f}$".format(loss_selected_k_4_free, loss_selected_pars_4_free[0], loss_selected_pars_4_free[1]))

ax3.legend()
ax3.set_xlabel("$N$ turns")
ax3.set_ylabel("$DA(N)$")
ax3.set_title("Comparison between original DA and DA obtained from real Loss,\n with Model 4 Fits $(\\lambda=0.5)$, 3 free parameters")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Comparison between original DA and DA obtained from real Loss,\n with Model 4 Fits $(\\lambda=0.5)$, 3 free parameters')

### Save data in the dataframe

In [28]:
fitting_data.loc[len(fitting_data)] = [
     "real",
     np.nan,
     real_selected_k_2,
     ks[1] - ks[0],
     real_selected_pars_2[0],
     np.sqrt(real_selected_co_pars_2[0, 0]), 
     real_selected_pars_2[1], 
     np.sqrt(real_selected_co_pars_2[1, 1]), 
     real_selected_err_2,
     real_selected_k_4, 
     ks[1] - ks[0], 
     real_selected_pars_4[0], 
     np.sqrt(real_selected_co_pars_4[0, 0]),
     1/2,
     real_selected_err_4,
     real_selected_k_4_free, 
     ks[1] - ks[0], 
     real_selected_pars_4_free[0], 
     np.sqrt(real_selected_co_pars_4_free[0, 0]),
     real_selected_pars_4_free[1], 
     np.sqrt(real_selected_co_pars_4_free[1, 1]),
     1/2,
     real_selected_err_4_free
]


fitting_data.loc[len(fitting_data)] = [
    "uniform",
     np.nan,
     loss_selected_k_2,
     ks[1] - ks[0],
     loss_selected_pars_2[0],
     np.sqrt(loss_selected_co_pars_2[0, 0]),
     loss_selected_pars_2[1],
     np.sqrt(loss_selected_co_pars_2[1, 1]),
     loss_selected_err_2,
     loss_selected_k_4,
     ks[1] - ks[0],
     loss_selected_pars_4[0],
     np.sqrt(loss_selected_co_pars_4[0, 0]),
     1/2,
     loss_selected_err_4,
     loss_selected_k_4_free,
     ks[1] - ks[0],
     loss_selected_pars_4_free[0],
     np.sqrt(loss_selected_co_pars_4_free[0, 0]),
     loss_selected_pars_4_free[1],
     np.sqrt(loss_selected_co_pars_4_free[1, 1]),
     1/2,
     loss_selected_err_4_free
]

## Loss and Fits -- Symmetric Gaussian Beam Distribution
### Choose your Sigma!

In [29]:
sigma = 8.0

### Loss comparison

In [30]:
# Assign uniform weights to engine and compute loss
engine.assign_weights(
    sx.assign_symmetric_gaussian(sigma)
)
real_values = engine.compute_loss(turn_sampling, DA0)
real_values = np.concatenate((real_values, [1.0]))

# Compute DA-based loss
values = symmetric_gaussian_loss(DA, sigma, DA0)
values = np.concatenate((values, [1.0]))

# Error computing
values1 = symmetric_gaussian_loss(DA - np.absolute(np.mean(radiuses - DA, axis=(0,1,2))), sigma, DA0)
values1 = np.concatenate((values1, [1.0]))

values2 = symmetric_gaussian_loss(DA + np.absolute(np.mean(radiuses - DA, axis=(0,1,2))), sigma, DA0)
values2 = np.concatenate((values2, [1.0]))

#### Visualize Loss comparison

In [31]:
fig2, ax2 = plt.subplots()
ax2.plot(axis_sampling, values, label="Values from DA")
ax2.fill_between(axis_sampling, values1, values2, label="Values from DA - error", color="C0", alpha=0.4)
ax2.plot(axis_sampling, real_values, label="Values from weights")
ax2.legend()
ax2.set_xlabel("$N$ turns")
ax2.set_ylabel("Active beam")
ax2.set_title("Symmetric Gaussian beam ($\\sigma={}$, Cutting Point at $DA=26.0$)".format(sigma))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Symmetric Gaussian beam ($\\sigma=8.0$, Cutting Point at $DA=26.0$)')

### Get DA from loss

In [32]:
DA_from_loss = DA_from_symmetric_gaussian_loss(real_values, sigma, DA0)

#### Visualize difference

In [33]:
fig3, ax3 = plt.subplots()
ax3.plot(turn_sampling, DA, label="Original DA")
ax3.plot(axis_sampling[:-1], DA_from_loss[:-1], label="DA extracted from real loss values")
ax3.legend()
ax3.set_xlabel("$N$ turns")
ax3.set_ylabel("$DA(N)$")
ax3.set_title("Comparison between original DA and DA obtained from real Loss")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Comparison between original DA and DA obtained from real Loss')

### Compare fitting values with Model 2

In [34]:
k_min = 0.05
k_max = 0.20
samples = 400

ks = np.linspace(k_min, k_max, samples)

real_pars, real_errs, real_co_pars = explore_k_model_2(turn_sampling, DA, k_min, k_max, samples)
loss_pars, loss_errs, loss_co_pars = explore_k_model_2(axis_sampling[:-1], DA_from_loss[:-1], k_min, k_max, samples)

real_selected_err_2 = np.min(real_errs)
real_selected_k_2 = ks[np.argmin(real_errs)]
real_selected_pars_2 = real_pars[np.argmin(real_errs)]
real_selected_co_pars_2 = real_co_pars[np.argmin(real_errs)]

loss_selected_err_2 = np.min(loss_errs)
loss_selected_k_2 = ks[np.argmin(loss_errs)]
loss_selected_pars_2 = loss_pars[np.argmin(loss_errs)]
loss_selected_co_pars_2 = loss_co_pars[np.argmin(loss_errs)]

In [35]:
print("REAL:", "$\\kappa:$", real_selected_k_2, ", $\\rho_\\ast$ and $N_0$", real_selected_pars_2)
print("FROM LOSS:", "$\\kappa:$", loss_selected_k_2, ", $\\rho_\\ast$ and $N_0$", loss_selected_pars_2)

REAL: $\kappa:$ 0.11992481203007521 , $\rho_\ast$ and $N_0$ [36.15442002 46.76235005]
FROM LOSS: $\kappa:$ 0.11729323308270678 , $\rho_\ast$ and $N_0$ [36.03135337 37.12036537]


In [36]:
fig3, ax3 = plt.subplots()

ax3.plot(turn_sampling, DA, label="Original DA")
ax3.plot(turn_sampling, model_2(turn_sampling, real_selected_pars_2[0], real_selected_pars_2[1], real_selected_k_2), label="Fitting on original DA\n$\\kappa = {:.4f},\\rho_\\ast = {:.4f}, N_0={:.4f}$".format(real_selected_k_2, real_selected_pars_2[0], real_selected_pars_2[1] ))

ax3.plot(axis_sampling[:-1], DA_from_loss[:-1], label="DA extracted from real loss values")
ax3.plot(turn_sampling, model_2(turn_sampling, loss_selected_pars_2[0], loss_selected_pars_2[1], loss_selected_k_2), label="Fitting on original DA\n$\\kappa = {:.4f},\\rho_\\ast = {:.4f}, N_0={:.4f}$".format(loss_selected_k_2, loss_selected_pars_2[0], loss_selected_pars_2[1] ))

ax3.legend()
ax3.set_xlabel("$N$ turns")
ax3.set_ylabel("$DA(N)$")
ax3.set_title("Comparison between original DA and DA obtained from real Loss, with Model 2 Fits")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Comparison between original DA and DA obtained from real Loss, with Model 2 Fits')

### Compare fitting values with Model 4

In [37]:
k_min = 0.15
k_max = 0.30
samples = 400

ks = np.linspace(k_min, k_max, samples)

real_pars, real_errs, real_co_pars = explore_k_model_4(turn_sampling, DA, k_min, k_max, samples)
loss_pars, loss_errs, loss_co_pars = explore_k_model_4(axis_sampling[:-1], DA_from_loss[:-1], k_min, k_max, samples)

real_selected_err_4 = np.min(real_errs)
real_selected_k_4 = ks[np.argmin(real_errs)]
real_selected_pars_4 = real_pars[np.argmin(real_errs)]
real_selected_co_pars_4 = real_co_pars[np.argmin(real_errs)]

loss_selected_err_4 = np.min(loss_errs)
loss_selected_k_4 = ks[np.argmin(loss_errs)]
loss_selected_pars_4 = loss_pars[np.argmin(loss_errs)]
loss_selected_co_pars_4 = loss_co_pars[np.argmin(loss_errs)]

In [38]:
print("REAL:", "$\\kappa:$", real_selected_k_4, ", $\\rho_\\ast$ and $N_0$", real_selected_pars_4)
print("FROM LOSS:", "$\\kappa:$", loss_selected_k_4, ", $\\rho_\\ast$ and $N_0$", loss_selected_pars_4)

REAL: $\kappa:$ 0.21616541353383456 , $\rho_\ast$ and $N_0$ [60.22905065]
FROM LOSS: $\kappa:$ 0.21729323308270676 , $\rho_\ast$ and $N_0$ [60.76538463]


In [39]:
fig3, ax3 = plt.subplots()

ax3.plot(turn_sampling, DA, label="Original DA")
ax3.plot(turn_sampling, model_4(turn_sampling, real_selected_pars_4[0], real_selected_k_4), label="Fitting on original DA\n$\\kappa = {:.4f},\\rho_\\ast = {:.4f}$".format(real_selected_k_4, real_selected_pars_4[0]))

ax3.plot(axis_sampling[:-1], DA_from_loss[:-1], label="DA extracted from real loss values")
ax3.plot(turn_sampling, model_4(turn_sampling, loss_selected_pars_4[0], loss_selected_k_4), label="Fitting on original DA\n$\\kappa = {:.4f},\\rho_\\ast = {:.4f}$".format(loss_selected_k_4, loss_selected_pars_4[0]))

ax3.legend()
ax3.set_xlabel("$N$ turns")
ax3.set_ylabel("$DA(N)$")
ax3.set_title("Comparison between original DA and DA obtained from real Loss, with Model 4 Fits $(\\lambda=0.5)$")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Comparison between original DA and DA obtained from real Loss, with Model 4 Fits $(\\lambda=0.5)$')

### Compare fitting values with Model 4 (But more free)

In [40]:
k_min = 0.05
k_max = 0.20
samples = 400

ks = np.linspace(k_min, k_max, samples)

real_pars, real_errs, real_co_pars = explore_model_4_free(turn_sampling, DA, k_min, k_max, samples)
loss_pars, loss_errs, loss_co_pars = explore_model_4_free(axis_sampling[:-1], DA_from_loss[:-1], k_min, k_max, samples)

real_selected_err_4_free = np.min(real_errs)
real_selected_k_4_free = ks[np.argmin(real_errs)]
real_selected_pars_4_free = real_pars[np.argmin(real_errs)]
real_selected_co_pars_4_free = real_co_pars[np.argmin(real_errs)]

loss_selected_err_4_free = np.min(loss_errs)
loss_selected_k_4_free = ks[np.argmin(loss_errs)]
loss_selected_pars_4_free = loss_pars[np.argmin(loss_errs)]
loss_selected_co_pars_4_free = loss_co_pars[np.argmin(loss_errs)]

In [41]:
print("REAL:", "$\\kappa:$", real_selected_k_4_free, ", $\\rho_\\ast$ and $N_0$", real_selected_pars_4_free)
print("FROM LOSS:", "$\\kappa:$", loss_selected_k_4_free, ", $\\rho_\\ast$ and $N_0$", loss_selected_pars_4_free)

REAL: $\kappa:$ 0.12293233082706768 , $\rho_\ast$ and $N_0$ [36.79845342 43.27224119]
FROM LOSS: $\kappa:$ 0.11992481203007521 , $\rho_\ast$ and $N_0$ [36.59997887 34.69222847]


In [42]:
fig3, ax3 = plt.subplots()

ax3.plot(turn_sampling, DA, label="Original DA")
ax3.plot(turn_sampling, model_4_free(turn_sampling, real_selected_pars_4_free[0], real_selected_pars_4_free[1], real_selected_k_4_free), label="Fitting on original DA\n$\\kappa = {:.4f},\\rho_\\ast = {:.4f}, N_0 = {:.4f}$".format(real_selected_k_4_free, real_selected_pars_4_free[0], real_selected_pars_4_free[1]))

ax3.plot(axis_sampling[:-1], DA_from_loss[:-1], label="DA extracted from real loss values")
ax3.plot(turn_sampling, model_4_free(turn_sampling, loss_selected_pars_4_free[0], loss_selected_pars_4_free[1], loss_selected_k_4_free), label="Fitting on original DA\n$\\kappa = {:.4f},\\rho_\\ast = {:.4f}, N_0 = {:.4f}$".format(loss_selected_k_4_free, loss_selected_pars_4_free[0], loss_selected_pars_4_free[1]))

ax3.legend()
ax3.set_xlabel("$N$ turns")
ax3.set_ylabel("$DA(N)$")
ax3.set_title("Comparison between original DA and DA obtained from real Loss,\n with Model 4 Fits $(\\lambda=0.5)$, 3 free parameters")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Comparison between original DA and DA obtained from real Loss,\n with Model 4 Fits $(\\lambda=0.5)$, 3 free parameters')

### Save data in the dataframe

In [43]:
fitting_data.loc[len(fitting_data)] = [
    "gaussian",
     sigma,
     loss_selected_k_2,
     ks[1] - ks[0],
     loss_selected_pars_2[0],
     np.sqrt(loss_selected_co_pars_2[0, 0]),
     loss_selected_pars_2[1],
     np.sqrt(loss_selected_co_pars_2[1, 1]),
     loss_selected_err_2,
     loss_selected_k_4,
     ks[1] - ks[0],
     loss_selected_pars_4[0],
     np.sqrt(loss_selected_co_pars_4[0, 0]),
     1/2,
     loss_selected_err_4,
     loss_selected_k_4_free,
     ks[1] - ks[0],
     loss_selected_pars_4_free[1],
     np.sqrt(loss_selected_co_pars_4_free[1, 1]),
     loss_selected_pars_4_free[0],
     np.sqrt(loss_selected_co_pars_4_free[0, 0]),
     1/2,
     loss_selected_err_4_free
]


## Display general dataframe with fitting data

In [44]:
fitting_data[["type", "sigma", "Model 2"]]

type sigma   Model 2                                            \
                          k     k err        rho   rho err         N0   
0      real   NaN  0.119925  0.000376  36.154420  0.003738  46.762350   
1   uniform   NaN  0.121429  0.000376  37.482097  0.009520  57.090429   
2  gaussian   8.0  0.117293  0.000376  36.031353  0.010386  37.120365   

                        
     N0 err       Chi2  
0  0.203332   0.008032  
1  0.543469  10.872958  
2  0.498382   0.208608

In [45]:
fitting_data[["type", "sigma", "Model 4 (2 free pars)"]]

type sigma Model 4 (2 free pars)                                        \
                                      k     k err        rho   rho err lambda   
0      real   NaN              0.216165  0.000376  60.229051  0.012648    0.5   
1   uniform   NaN              0.223684  0.000376  64.376111  0.020638    0.5   
2  gaussian   8.0              0.217293  0.000376  60.765385  0.015983    0.5   

              
        Chi2  
0   0.180550  
1  11.032593  
2   0.323162

In [46]:
fitting_data[["type", "sigma", "Model 4 (3 free pars)"]]

type sigma Model 4 (3 free pars)                                 \
                                      k     k err        rho   rho err   
0      real   NaN              0.122932  0.000376  36.798453  0.003951   
1   uniform   NaN              0.124060  0.000376  38.085759  0.009677   
2  gaussian   8.0              0.119925  0.000376  34.692228  0.473111   

                                          
          N0    N0 err lambda       Chi2  
0  43.272241  0.194533    0.5   0.008776  
1  52.815583  0.497892    0.5  10.872947  
2  36.599979  0.010746    0.5   0.208704

# Fokker-Planck and Nekhoroshev

In [47]:
import crank_nicolson_numba.nekhoroshev as nk

In [48]:
k = fitting_data[("Model 4 (3 free pars)", "k")][0]
rho = fitting_data[("Model 4 (3 free pars)", "rho")][0]
N0 = fitting_data[("Model 4 (3 free pars)", "N0")][0]

## 4D normal distribution in 1D Action variable
If we consider the following distribution:

$$ f(x,p_x,y,p_y) = \frac{1}{4\pi^2\sigma^4}\exp\left[-\frac{1}{2}\left(\frac{x^2}{\sigma^2}+\frac{p_x^2}{\sigma^2}+\frac{y^2}{\sigma^2}+\frac{p_y^2}{\sigma^2}\right)\right]$$

we convert the distribution to polar variables:

$$f(r, \alpha, \theta_1, \theta_2) = \frac{r^3 \sin(\alpha)\cos(\alpha)}{4\pi^2\sigma^4} \exp\left[-\frac{1}{2}\left(\frac{r^2}{\sigma^2}\right)\right]$$

we then consider distribution integrated over $\alpha, \theta_1, \theta_2$:

$$f(r) = \int_0^{2\pi}\int_0^{2\pi}\int_0^{\pi/2} f(r,\alpha,\theta_1,\theta_2)\,d\alpha\,d\theta_1\,d\theta_2 = \frac{r^3}{2\sigma^4}\exp\left[-\frac{1}{2}\left(\frac{r^2}{\sigma^2}\right)\right]$$


and then we consider $I = r^2/2$ which implies also $dr = dI/I^{1/2}$, and end up with:

$$f(I) = \frac{I}{\sigma^4}\exp\left[-\frac{I}{\sigma^2}\right] = c I \exp\left[-\frac{I}{\sigma^2}\right]$$

where $c$ will be normalized in order to have a unitary starting value for the distributions considered.

## Nekhoroshev-like Diffusion equation

$$D(I) = c \exp\left[-\left(\frac{I_\ast}{I}\right)^{1/2\kappa}\right]$$

$$D(I) = c \exp\left[-2\left(\frac{I_\ast}{I}\right)^{1/2\kappa}\right]$$

(verificare su articolo)

* Try to use free $\kappa$
* (done) Fix $\kappa$ 

### Parameters setup

In [49]:
I_max = DA0**2 / 2
I, stepI = np.linspace(0, I_max, 3000, retstep=True)
I0 = I * np.exp(-(I/sigma**2))

I0 = nk.logistic_damping(I, I0, 470, 10)

I0 /= integrate.trapz(I0, I)

c = 1.0

I_star_min = 150.0
I_star_max = 350.0
I_star_steps = 401
I_star_list = np.linspace(I_star_min, I_star_max, I_star_steps)

iter_step = 100

In [50]:
t1_fig, t1_ax = plt.subplots()
t1_ax.plot(I, I0)
t1_ax.set_xlabel("$I$")
t1_ax.set_ylabel("$f(I)$")
t1_ax.set_title("Starting distribution for FP (logistic dumping added at the end)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Starting distribution for FP (logistic dumping added at the end)')

In [51]:
data = []
errors = []
original_times = []
sanity = []

dt = 1.0
c = 1.0
for j, I_star in tqdm(enumerate(I_star_list), total=I_star_steps):
    engine_nk = nk.cn_nekhoroshev(I_max, c, I_star, 1 / (k * 2), 0, I0, dt)
    i = 0
    
    t = []
    survival = []

    while(1.0 - engine_nk.get_particle_loss() >= values[0]):
        t.append(i * iter_step * dt)
        survival.append(1.0 - engine_nk.get_particle_loss())
        engine_nk.iterate(iter_step)
        i += 1
    t.append(i * iter_step * dt)
    survival.append(1.0 - engine_nk.get_particle_loss())

    print("Step:", j, "/", I_star_steps, ",", "Number of steps executed", i, ", dt:", dt)
    if i > 500:
        print("Increasing dt!")
        dt *= 2
        
    
    f = interp1d(t, survival, kind="cubic")
    inv_f = inversefunc(f, domain=(t[0], t[-1]))
    point_t = inv_f(values[0])
    points_t = np.linspace(0, point_t, len(axis_sampling))
    values_f = f(points_t)
    values_f = values_f[::-1]
    
    data.append(values_f)
    errors.append(chi_2(values, values_f))
    original_times.append(np.asarray(t)[::-1])
    sanity.append(engine_nk.get_sanity())
    
    if np.count_nonzero(values_f - values < 0.0) >= len(values_f) - 6:
        print("We went down now!")
        break

Step: 0 / 401 , Number of steps executed 1501 , dt: 1.0
Increasing dt!
Step: 1 / 401 , Number of steps executed 754 , dt: 2.0
Increasing dt!
Step: 2 / 401 , Number of steps executed 379 , dt: 4.0
Step: 3 / 401 , Number of steps executed 381 , dt: 4.0
Step: 4 / 401 , Number of steps executed 383 , dt: 4.0
Step: 5 / 401 , Number of steps executed 384 , dt: 4.0
Step: 6 / 401 , Number of steps executed 386 , dt: 4.0
Step: 7 / 401 , Number of steps executed 388 , dt: 4.0
Step: 8 / 401 , Number of steps executed 390 , dt: 4.0
Step: 9 / 401 , Number of steps executed 391 , dt: 4.0
Step: 10 / 401 , Number of steps executed 393 , dt: 4.0
Step: 11 / 401 , Number of steps executed 395 , dt: 4.0
Step: 12 / 401 , Number of steps executed 397 , dt: 4.0
Step: 13 / 401 , Number of steps executed 399 , dt: 4.0
Step: 14 / 401 , Number of steps executed 401 , dt: 4.0
Step: 15 / 401 , Number of steps executed 403 , dt: 4.0
Step: 16 / 401 , Number of steps executed 405 , dt: 4.0
Step: 17 / 401 , Number of 

In [52]:
best_idx = np.argmin(errors)
best_data = data[best_idx]
best_time = original_times[best_idx]
best_I_star = I_star_list[best_idx]

In [53]:
print("Displaying the best FP values obtained:")
print("I_star = {}".format(best_I_star))
print("Time scale = {}".format(best_time[0]/axis_sampling[0]))

Displaying the best FP values obtained:
I_star = 259.5
Time scale = 416.768


In [54]:
fp2_fig, fp2_ax = plt.subplots()
fp2_ax.plot(axis_sampling, values, label="Loss from original DA")
fp2_ax.plot(axis_sampling, best_data, label="FP process $(I_\\ast={})$".format(best_I_star))
fp2_ax.legend()
fp2_ax.set_xlabel("$N$ turns")
fp2_ax.set_ylabel("Active beam")
fp2_ax.set_title("Comparison between loss curve from DA\nand best equivalent Fokker-Planck process $(\\kappa={:.3}, I_\\ast={})$".format(k, best_I_star))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Comparison between loss curve from DA\nand best equivalent Fokker-Planck process $(\\kappa=0.123, I_\\ast=259.5)$')

In [55]:
fp3_fig, fp3_ax = plt.subplots()
fp3_ax.plot(I_star_list[:len(errors)], errors)
fp3_ax.set_xlabel("$I_\\ast$ value")
fp3_ax.set_ylabel("Error value [A.U.]")
fp3_ax.set_title("Error measured from the various FP loss comparisons performed")

#fp3_ax.plot(I_star_list[:len(sanity)], np.asarray(sanity))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Error measured from the various FP loss comparisons performed')

In [56]:
fp1_fig, fp1_ax = plt.subplots()
cmap = matplotlib.cm.get_cmap('viridis')
for i, d in enumerate(data):
    fp1_ax.plot(axis_sampling, d, c=cmap(i/(len(data)-1)))
fp1_ax.plot(axis_sampling, values, c="red", label="Loss from original DA")
fp1_ax.legend()
fp1_ax.set_xlabel("$N$ turns")
fp1_ax.set_ylabel("Active Beam")
fp1_ax.set_title("Explored $I_\\ast$ values")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Explored $I_\\ast$ values')

## Test with also $\kappa$ somewhat free

In [89]:
k = fitting_data[("Model 4 (3 free pars)", "k")][0]
rho = fitting_data[("Model 4 (3 free pars)", "rho")][0]
N0 = fitting_data[("Model 4 (3 free pars)", "N0")][0]

In [108]:
I_max = DA0**2 / 2
I, stepI = np.linspace(0, I_max, 3000, retstep=True)
I0 = I * np.exp(-(I/sigma**2))

I0 = nk.logistic_damping(I, I0, 470, 15)

I0 /= integrate.trapz(I0, I)

c = 1.0

I_star_min = 250.0
I_star_max = 450.0
I_star_steps = 101
I_star_list = np.linspace(I_star_min, I_star_max, I_star_steps)

iter_step = 100

dk = 0.1
k_list = np.linspace(k, k + dk, 100)

In [109]:
best_data = []
best_errors = []
best_time = []
best_I_star = []
best_data = []

for k_index, now_k in tqdm(list(enumerate(k_list))):

    data = []
    errors = []
    original_times = []
    sanity = []

    dt = 16.0
    c = 1.0
    for j, I_star in tqdm(enumerate(I_star_list), total=I_star_steps):
        engine_nk = nk.cn_nekhoroshev(I_max, c, I_star, 1 / (now_k * 2), 0, I0, dt)
        i = 0

        t = []
        survival = []

        while(1.0 - engine_nk.get_particle_loss() >= values[0]):
            t.append(i * iter_step * dt)
            survival.append(1.0 - engine_nk.get_particle_loss())
            engine_nk.iterate(iter_step)
            i += 1
        t.append(i * iter_step * dt)
        survival.append(1.0 - engine_nk.get_particle_loss())

        print("Step:", j, "/", I_star_steps, ",", "Number of steps executed", i, ", dt:", dt)
        if i > 500:
            print("Increasing dt!")
            dt *= 2


        f = interp1d(t, survival, kind="cubic")
        inv_f = inversefunc(f, domain=(t[0], t[-1]))
        point_t = inv_f(values[0])
        points_t = np.linspace(0, point_t, len(axis_sampling))
        values_f = f(points_t)
        values_f = values_f[::-1]

        data.append(values_f)
        errors.append(chi_2(values, values_f))
        original_times.append(np.asarray(t)[::-1])
        sanity.append(engine_nk.get_sanity())

        if np.count_nonzero(values_f - values < 0.0) >= len(values_f) - 6:
            print("We went down now!")
            break
    
    best_idx = np.argmin(errors)
    best_errors.append(np.min(errors))
    best_data.append(data[best_idx])
    best_time.append(original_times[best_idx])
    best_I_star.append(I_star_list[best_idx])

Step: 0 / 101 , Number of steps executed 3277 , dt: 16.0
Increasing dt!
Step: 1 / 101 , Number of steps executed 2145 , dt: 32.0
Increasing dt!
Step: 2 / 101 , Number of steps executed 1422 , dt: 64.0
Increasing dt!
Step: 3 / 101 , Number of steps executed 955 , dt: 128.0
Increasing dt!
Step: 4 / 101 , Number of steps executed 650 , dt: 256.0
Increasing dt!
Step: 5 / 101 , Number of steps executed 447 , dt: 512.0
Step: 6 / 101 , Number of steps executed 623 , dt: 512.0
Increasing dt!
Step: 7 / 101 , Number of steps executed 440 , dt: 1024.0
We went down now!



Step: 0 / 101 , Number of steps executed 2980 , dt: 16.0
Increasing dt!
Step: 1 / 101 , Number of steps executed 1934 , dt: 32.0
Increasing dt!
Step: 2 / 101 , Number of steps executed 1271 , dt: 64.0
Increasing dt!
Step: 3 / 101 , Number of steps executed 846 , dt: 128.0
Increasing dt!
Step: 4 / 101 , Number of steps executed 570 , dt: 256.0
Increasing dt!
Step: 5 / 101 , Number of steps executed 389 , dt: 512.0
Step: 6 / 101 , Number of steps executed 536 , dt: 512.0
Increasing dt!
Step: 7 / 101 , Number of steps executed 374 , dt: 1024.0
Step: 8 / 101 , Number of steps executed 529 , dt: 1024.0
Increasing dt!
We went down now!



Step: 0 / 101 , Number of steps executed 2722 , dt: 16.0
Increasing dt!
Step: 1 / 101 , Number of steps executed 1751 , dt: 32.0
Increasing dt!
Step: 2 / 101 , Number of steps executed 1141 , dt: 64.0
Increasing dt!
Step: 3 / 101 , Number of steps executed 753 , dt: 128.0
Increasing dt!
Step: 4 / 101 , Number of steps executed 503 , dt: 256.0
Increasing dt!
Step: 5 / 101 , Number of steps executed 340 , dt: 512.0
Step: 6 / 101 , Number of steps executed 464 , dt: 512.0
Step: 7 / 101 , Number of steps executed 642 , dt: 512.0
Increasing dt!
Step: 8 / 101 , Number of steps executed 449 , dt: 1024.0
We went down now!



Step: 0 / 101 , Number of steps executed 2497 , dt: 16.0
Increasing dt!
Step: 1 / 101 , Number of steps executed 1593 , dt: 32.0
Increasing dt!
Step: 2 / 101 , Number of steps executed 1029 , dt: 64.0
Increasing dt!
Step: 3 / 101 , Number of steps executed 673 , dt: 128.0
Increasing dt!
Step: 4 / 101 , Number of steps executed 446 , dt: 256.0
Step: 5 / 101 , Number of steps executed 597 , dt: 256.0
Increasing dt!
Step: 6 / 101 , Number of steps executed 404 , dt: 512.0
Step: 7 / 101 , Number of steps executed 553 , dt: 512.0
Increasing dt!
Step: 8 / 101 , Number of steps executed 383 , dt: 1024.0
Step: 9 / 101 , Number of steps executed 537 , dt: 1024.0
Increasing dt!
We went down now!



Step: 0 / 101 , Number of steps executed 2300 , dt: 16.0
Increasing dt!
Step: 1 / 101 , Number of steps executed 1456 , dt: 32.0
Increasing dt!
Step: 2 / 101 , Number of steps executed 933 , dt: 64.0
Increasing dt!
Step: 3 / 101 , Number of steps executed 605 , dt: 128.0
Increasing dt!
Step: 4 / 101 , Number of steps executed 397 , dt: 256.0
Step: 5 / 101 , Number of steps executed 527 , dt: 256.0
Increasing dt!
Step: 6 / 101 , Number of steps executed 354 , dt: 512.0
Step: 7 / 101 , Number of steps executed 480 , dt: 512.0
Step: 8 / 101 , Number of steps executed 659 , dt: 512.0
Increasing dt!
Step: 9 / 101 , Number of steps executed 457 , dt: 1024.0
We went down now!



...

Step: 0 / 101 , Number of steps executed 391 , dt: 16.0
Step: 1 / 101 , Number of steps executed 406 , dt: 16.0
Step: 2 / 101 , Number of steps executed 422 , dt: 16.0
Step: 3 / 101 , Number of steps executed 439 , dt: 16.0
Step: 4 / 101 , Number of steps executed 457 , dt: 16.0
Step: 5 / 101 , Number of steps executed 476 , dt: 16.0
Step: 6 / 101 , Number of steps executed 497 , dt: 16.0
Step: 7 / 101 , Number of steps executed 518 , dt: 16.0
Increasing dt!
Step: 8 / 101 , Number of steps executed 271 , dt: 32.0
Step: 9 / 101 , Number of steps executed 283 , dt: 32.0
Step: 10 / 101 , Number of steps executed 297 , dt: 32.0
Step: 11 / 101 , Number of steps executed 311 , dt: 32.0
Step: 12 / 101 , Number of steps executed 326 , dt: 32.0
Step: 13 / 101 , Number of steps executed 342 , dt: 32.0
Step: 14 / 101 , Number of steps executed 360 , dt: 32.0
Step: 15 / 101 , Number of steps executed 379 , dt: 32.0
Step: 16 / 101 , Number of steps executed 399 , dt: 32.0
Step: 17 / 101 , Number of

Step: 0 / 101 , Number of steps executed 390 , dt: 16.0
Step: 1 / 101 , Number of steps executed 405 , dt: 16.0
Step: 2 / 101 , Number of steps executed 421 , dt: 16.0
Step: 3 / 101 , Number of steps executed 437 , dt: 16.0
Step: 4 / 101 , Number of steps executed 455 , dt: 16.0
Step: 5 / 101 , Number of steps executed 474 , dt: 16.0
Step: 6 / 101 , Number of steps executed 494 , dt: 16.0
Step: 7 / 101 , Number of steps executed 515 , dt: 16.0
Increasing dt!
Step: 8 / 101 , Number of steps executed 269 , dt: 32.0
Step: 9 / 101 , Number of steps executed 282 , dt: 32.0
Step: 10 / 101 , Number of steps executed 295 , dt: 32.0
Step: 11 / 101 , Number of steps executed 309 , dt: 32.0
Step: 12 / 101 , Number of steps executed 323 , dt: 32.0
Step: 13 / 101 , Number of steps executed 339 , dt: 32.0
Step: 14 / 101 , Number of steps executed 357 , dt: 32.0
Step: 15 / 101 , Number of steps executed 375 , dt: 32.0
Step: 16 / 101 , Number of steps executed 395 , dt: 32.0
Step: 17 / 101 , Number of

Step: 0 / 101 , Number of steps executed 389 , dt: 16.0
Step: 1 / 101 , Number of steps executed 403 , dt: 16.0
Step: 2 / 101 , Number of steps executed 419 , dt: 16.0
Step: 3 / 101 , Number of steps executed 436 , dt: 16.0
Step: 4 / 101 , Number of steps executed 453 , dt: 16.0
Step: 5 / 101 , Number of steps executed 472 , dt: 16.0
Step: 6 / 101 , Number of steps executed 492 , dt: 16.0
Step: 7 / 101 , Number of steps executed 513 , dt: 16.0
Increasing dt!
Step: 8 / 101 , Number of steps executed 268 , dt: 32.0
Step: 9 / 101 , Number of steps executed 280 , dt: 32.0
Step: 10 / 101 , Number of steps executed 293 , dt: 32.0
Step: 11 / 101 , Number of steps executed 306 , dt: 32.0
Step: 12 / 101 , Number of steps executed 321 , dt: 32.0
Step: 13 / 101 , Number of steps executed 337 , dt: 32.0
Step: 14 / 101 , Number of steps executed 353 , dt: 32.0
Step: 15 / 101 , Number of steps executed 371 , dt: 32.0
Step: 16 / 101 , Number of steps executed 391 , dt: 32.0
Step: 17 / 101 , Number of

Step: 0 / 101 , Number of steps executed 387 , dt: 16.0
Step: 1 / 101 , Number of steps executed 402 , dt: 16.0
Step: 2 / 101 , Number of steps executed 417 , dt: 16.0
Step: 3 / 101 , Number of steps executed 434 , dt: 16.0
Step: 4 / 101 , Number of steps executed 451 , dt: 16.0
Step: 5 / 101 , Number of steps executed 469 , dt: 16.0
Step: 6 / 101 , Number of steps executed 489 , dt: 16.0
Step: 7 / 101 , Number of steps executed 510 , dt: 16.0
Increasing dt!
Step: 8 / 101 , Number of steps executed 266 , dt: 32.0
Step: 9 / 101 , Number of steps executed 278 , dt: 32.0
Step: 10 / 101 , Number of steps executed 291 , dt: 32.0
Step: 11 / 101 , Number of steps executed 304 , dt: 32.0
Step: 12 / 101 , Number of steps executed 318 , dt: 32.0
Step: 13 / 101 , Number of steps executed 334 , dt: 32.0
Step: 14 / 101 , Number of steps executed 350 , dt: 32.0
Step: 15 / 101 , Number of steps executed 368 , dt: 32.0
Step: 16 / 101 , Number of steps executed 387 , dt: 32.0
Step: 17 / 101 , Number of

Step: 0 / 101 , Number of steps executed 386 , dt: 16.0
Step: 1 / 101 , Number of steps executed 401 , dt: 16.0
Step: 2 / 101 , Number of steps executed 416 , dt: 16.0
Step: 3 / 101 , Number of steps executed 432 , dt: 16.0
Step: 4 / 101 , Number of steps executed 449 , dt: 16.0
Step: 5 / 101 , Number of steps executed 467 , dt: 16.0
Step: 6 / 101 , Number of steps executed 487 , dt: 16.0
Step: 7 / 101 , Number of steps executed 507 , dt: 16.0
Increasing dt!
Step: 8 / 101 , Number of steps executed 265 , dt: 32.0
Step: 9 / 101 , Number of steps executed 276 , dt: 32.0
Step: 10 / 101 , Number of steps executed 289 , dt: 32.0
Step: 11 / 101 , Number of steps executed 302 , dt: 32.0
Step: 12 / 101 , Number of steps executed 316 , dt: 32.0
Step: 13 / 101 , Number of steps executed 331 , dt: 32.0
Step: 14 / 101 , Number of steps executed 347 , dt: 32.0
Step: 15 / 101 , Number of steps executed 365 , dt: 32.0
Step: 16 / 101 , Number of steps executed 383 , dt: 32.0
Step: 17 / 101 , Number of

In [110]:
fpk1_fig, fpk1_ax = plt.subplots()

fpk1_ax.scatter(k_list, best_I_star)
fpk1_ax.set_xlabel("$\\kappa$ value")
fpk1_ax.set_ylabel("$I_\\ast$ value")

fpk1_ax.set_title("FP fitting values obtained for different $\\kappa$ values")

/home/carlidel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'FP fitting values obtained for different $\\kappa$ values')

In [124]:
fpk2_fig, fpk2_ax = plt.subplots()

fpk2_ax.set_xlabel("$\\kappa$ value")
fpk2_ax.set_ylabel("$\\chi^2$ error")
fpk2_ax.scatter(k_list, best_errors, s=10.0, label="FP samples")

fpk2_ax.axvline(k, c="red", label="Best fit from Model 4 $(\\kappa={:.3})$".format(k))
fpk2_ax.axvline(k_list[np.argmin(best_errors)], c="green", label="FP best $(\\kappa={:.3})$".format(k_list[np.argmin(best_errors)]))

fpk2_ax.legend()

fpk2_ax.set_title("Error obtained for different $\\kappa$ values in FP fitting process")

/home/carlidel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Error obtained for different $\\kappa$ values in FP fitting process')

In [126]:
fpk3_fig, fpk3_ax = plt.subplots()
cmap = matplotlib.cm.get_cmap('viridis')
for i, d in enumerate(best_data):
    fpk3_ax.plot(axis_sampling, d, c=cmap(i/(len(best_data)-1)))
fpk3_ax.plot(axis_sampling, values, c="red", label="Loss from original DA")
fpk3_ax.legend()
fpk3_ax.set_xlabel("$N$ turns")
fpk3_ax.set_ylabel("Active Beam")
fpk3_ax.set_title("Explored $\\kappa$ values")

/home/carlidel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Explored $\\kappa$ values')

In [128]:
fpk4_fig, fpk4_ax = plt.subplots()
fpk4_ax.plot(axis_sampling, values, label="Loss from original DA")
fpk4_ax.plot(axis_sampling, best_data[np.argmin(best_errors)], label="Best FP fit")
fpk4_ax.legend()
fpk4_ax.set_xlabel("$N$ turns")
fpk4_ax.set_ylabel("Active Beam")
fpk4_ax.set_title("Best Fokker-Planck fit (free $\\kappa$), $\\kappa={:.3}, I_\\ast={:.3}$".format(k_list[np.argmin(best_errors)], best_I_star[np.argmin(best_errors)]))

/home/carlidel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Best Fokker-Planck fit (free $\\kappa$), $\\kappa=0.175, I_\\ast=3.24e+02$')